# Feature selection strategies

In this notebook we train models using the 57 parameters available. Then, we select the most important parameters through Strategy A, B, and C.

This require high computational power. The entire notebook, including model fitting and feature selection, can be run in about 20-48 hours using a 16-core CPU. Set `run = True` to run the notebook including intensive task. Otherwise, set `run = False` to use previously saved work and solely plot results.

In [1]:
# Run intensive tasks
run = False

import aidxmods as axm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Full parameter models

In [2]:
# Base model
if run:
    voyager = (
        axm.get(name='voyager')
        .preprocess()
        .fitmodel()
    )
    
else:
    voyager = axm.load('voyager')

print(voyager)
print_ap = lambda x: f'{x[0]:.3f} [{x[1]:.3f} - {x[2]:.3f}]'

voyager_train = voyager.ap_prc(dataset='train')
voyager_test = voyager.ap_prc(dataset='test')
voyager_val = voyager.ap_prc(dataset='val')
print(f'{"MLC":21}', f'{"Training AP":21}', f'{"Test AP":21}', f'{"Validation AP":21}', sep=' | ')
print(f'{"":-<21}', f'{"":-<21}', f'{"":-<21}', f'{"":-<21}', sep=' | ')
for estimator in voyager.available_estimators.values():
    print(f'{estimator:21}', print_ap(voyager_train[estimator]), print_ap(voyager_test[estimator]), print_ap(voyager_val[estimator]), sep=' | ')

Fitter name: voyager
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 57
Features used: 98
MLC                   | Training AP           | Test AP               | Validation AP        
--------------------- | --------------------- | --------------------- | ---------------------
XGBoost               | 0.380 [0.361 - 0.398] | 0.297 [0.261 - 0.334] | 0.252 [0.239 - 0.268]
Random Forest         | 0.298 [0.282 - 0.316] | 0.213 [0.185 - 0.245] | 0.210 [0.199 - 0.225]
Logistic Regression   | 0.171 [0.158 - 0.184] | 0.163 [0.139 - 0.192] | 0.203 [0.191 - 0.218]
Naive Bayes           | 0.135 [0.127 - 0.143] | 0.136 [0.119 - 0.153] | 0.151 [0.144 - 0.160]
SOFA                  | 0.079 [0.074 - 0.083] | 0.076 [0.068 - 0.085] | 0.152 [0.144 - 0.160]
OASIS                 | 0.134 [0.125 - 0.144] | 0.138 [0.121 - 0.162] | 0.170 [0.161 - 0.181]
MODS                  | 0.094 [0.088 - 0.102] | 0.089 [0.078 - 0.103] 

# Strategy A

In [3]:
# Base model
if run:
    strata = (
        axm.get(name='strata')
        .preprocess()
        .fitmodel()
    )
    strata = axm.rfe(strata, n=22, step=1, n_iter=5, kfold=10).save()
    
else:
    strata = axm.load('strata')
    
# Select most important features based on RFE
mif = axm.most_important_features(strata)

# Make a dict of fitters with most important features
strata20 = dict()
for key, features in mif.items():
    if run:
        strata20[key] = (
            axm.get(name='strata20' + key)
            .preprocess(restrict=features)
            .fitmodel(key)
            .save()
        )

    else:
        strata20[key] = axm.load('strata20' + key)
        
    print(strata20[key], end='\n\n')

Fitter name: strata20xg
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 37

Fitter name: strata20rf
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 19
Features used: 37

Fitter name: strata20lr
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 36

Fitter name: strata20nb
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 33



# Strategy B

In [4]:
# Base model
if run:
    stratb = (
        axm.get(name='stratb')
        .preprocess(restrict=list(axm.get_features('base', '!missingFlag', 'enabledToFit')))
        .fitmodel()
        .save()
    )
    stratb = axm.rfe(stratb, n=22, step=1, n_iter=5, kfold=10).save()
    
else:
    stratb = axm.load('stratb')
    
# Select most important features based on RFE
mif = axm.most_important_features(stratb)

# Make a dict of fitters with most important features
stratb20 = dict()
for key, features in mif.items():
    if run:
        stratb20[key] = (
            axm.get(name='stratb20' + key)
            .preprocess(restrict=features)
            .fitmodel(key)
            .save()
        )

    else:
        stratb20[key] = axm.load('stratb20' + key)
        
    print(stratb20[key], end='\n\n')

Fitter name: stratb20xg
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 37

Fitter name: stratb20rf
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 39

Fitter name: stratb20lr
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 31

Fitter name: stratb20nb
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 20
Features used: 31



# Strategy C

In [5]:
# Base model
if run:
    stratc = (
        axm.get(name='stratc')
        .preprocess(restrict=["Age", "Age Miss" "Heart rate", "Heart rate Miss", "SBP", "SBP Miss", 
                              "Temperature", "Temperatur Miss", "GCS", "GCS Miss", "PAO2", "PAO2 Miss", 
                              "FIO2", "FIO2 Miss", "BUN", "BUN Miss", "Urine output", "Urine Output Miss", 
                              "Sodium", "Sodium Miss", "Potassium", "Potassium Miss", "Bicarbonate", "Bicarbonate Miss", 
                              "Bilirubin", "Bilirubin Miss", "WBC", "WBC Miss", "Ventilation", "Cancer", 
                              "Lymphoma", "AIDS", "Admission for surgery", "Adm. for elec. surgery"])
        .fitmodel()
        .save()
    )
    
else:
    stratc = axm.load('stratc')
    
print(stratc)

Fitter name: stratc
Total dataset size: 112,023
Training dataset size: 65,880
Testing dataset size: 16,471
Validation dataset size: 29,672
Parameters used: 19
Features used: 30


# Summary

In [ ]:
strata_fimp = pd.DataFrame([
    strata20['xg'].get_feature_importance('xg'),
    strata20['rf'].get_feature_importance('rf'),
    strata20['lr'].get_feature_importance('lr'),
    strata20['nb'].get_feature_importance('nb')
])

stratb_fimp = pd.DataFrame([
    stratb20['xg'].get_feature_importance('xg'),
    stratb20['rf'].get_feature_importance('rf'),
    stratb20['lr'].get_feature_importance('lr'),
    stratb20['nb'].get_feature_importance('nb')
])

stratc_fimp = pd.DataFrame([
    stratc.get_feature_importance('xg'),
    stratc.get_feature_importance('rf'),
    stratc.get_feature_importance('lr'),
    stratc.get_feature_importance('nb')
])

d = {'Strategy A' : strata_fimp, 'Strategy B' : stratb_fimp, 'Strategy C': stratc_fimp}
summary_fimp = pd.concat(d.values(), axis=0, keys=d.keys()).T
summary_fimp.to_csv('data/summary-fimportances.csv')
print('feature importance summary')
summary_fimp

In [ ]:
mlc = ['xg', 'rf', 'lr', 'nb']
MLC = ['XGBoost', 'Random Forest', 'Logistic Regression', 'Naive Bayes']
cus = ['saps', 'apache', 'mods', 'oasis']
CUS = ['SAPS II', 'APACHE II', 'MODS', 'OASIS']

a1, a2, a3 = dict(), dict(), dict()
b1, b2, b3 = dict(), dict(), dict()
c1, c2, c3 = dict(), dict(), dict()

for key in mlc:
    estimator_name = axm.Fitter.available_estimators[key]
    
    fitter = axm.load('strata20' + key)
    a1[estimator_name] = fitter.ap_prc(key, 'train')[estimator_name]
    a2[estimator_name] = fitter.ap_prc(key, 'test')[estimator_name]
    a3[estimator_name] = fitter.ap_prc(key, 'val')[estimator_name]
    
    fitter = axm.load('stratb20' + key)
    b1[estimator_name] = fitter.ap_prc(key, 'train')[estimator_name]
    b2[estimator_name] = fitter.ap_prc(key, 'test')[estimator_name]
    b3[estimator_name] = fitter.ap_prc(key, 'val')[estimator_name]

stratc = axm.load('stratc')
c1 = stratc.ap_prc(mlc + cus, 'train')
c2 = stratc.ap_prc(mlc + cus, 'test')
c3 = stratc.ap_prc(mlc + cus, 'val')

for key in cus:
    estimator_name = stratc.available_estimators[key]
    
    a1[estimator_name] = c1[estimator_name]
    a2[estimator_name] = c2[estimator_name]
    a3[estimator_name] = c3[estimator_name]
    
    b1[estimator_name] = c1[estimator_name]
    b2[estimator_name] = c2[estimator_name]
    b3[estimator_name] = c3[estimator_name]
    
columns = ["Training", "Test", "Validation"]
index = pd.MultiIndex.from_tuples(
    [
        ["Strategy A", "XGBoost"],
        ["Strategy A", "Random Forest"],
        ["Strategy A", "Logistic Regression"],
        ["Strategy A", "Naive Bayes"],
        ["Strategy B", "XGBoost"],
        ["Strategy B", "Random Forest"],
        ["Strategy B", "Logistic Regression"],
        ["Strategy B", "Naive Bayes"],
        ["Strategy C", "XGBoost"],
        ["Strategy C", "Random Forest"],
        ["Strategy C", "Logistic Regression"],
        ["Strategy C", "Naive Bayes"],
        ["CUS", "SAPS II"],
        ["CUS", "APACHE II"],
        ["CUS", "MODS"],
        ["CUS", "OASIS"],
    ], names=["Group", "Estimator"])
summary_perf = pd.DataFrame(columns=columns, index=index)
print_ap = lambda x: f'{x[0]:.3f} [{x[1]:.3f} - {x[2]:.3f}]'

In [ ]:
estimators = MLC + CUS
groups = ['Training', 'Test', 'Validation']
N = len(estimators)
width = 0.8
color_cycle = axm.GS_color_cycle[:4] * 2
hatch_cycle = [''] * 4 + ['///'] * 4
    
fig, ax = axm.setfig(1, 1, nseries=len(estimators), figsize=(7, 3))

for i, estimator in enumerate(estimators):
    offset = axm.get_barplot_offset(i, N)
    xpos = np.arange(len(groups)) + width * offset
    
    mean1, lower1, upper1 = a1[estimator]
    mean2, lower2, upper2 = a2[estimator]
    mean3, lower3, upper3 = a3[estimator]
    yerr = [[mean1 - lower1, mean2 - lower2, mean3 - lower3], [upper1 - mean1, upper2 - mean2, upper3 - mean3]]
    
    ax.bar(
        xpos, [mean1, mean2, mean3], width / N, 
        yerr=yerr, capsize=3, label=estimator, 
        color=color_cycle[i], edgecolor='#ffffff', hatch=hatch_cycle[i]
    )
    ax.bar(xpos, [mean1, mean2, mean3], width / N, color='none', edgecolor='k', zorder=1)
    ax.set_ylabel('Average precision')
    ax.set_ylim(0, 0.45)
    ax.set_xticks(np.arange(len(groups)))
    ax.set_xticklabels(groups)
    ax.grid(True, axis='y')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1, 0, 0))
    
    if estimator in MLC:
        summary_perf.loc[('Strategy A', estimator)] = (
            print_ap(a1[estimator]),
            print_ap(a2[estimator]),
            print_ap(a3[estimator])
        )
    
plt.tight_layout()
axm.handlefig('2-performance-summary-strata')

fig, ax = axm.setfig(1, 1, nseries=len(estimators), figsize=(7, 3))

for i, estimator in enumerate(estimators):
    offset = axm.get_barplot_offset(i, N)
    xpos = np.arange(len(groups)) + width * offset
    
    mean1, lower1, upper1 = b1[estimator]
    mean2, lower2, upper2 = b2[estimator]
    mean3, lower3, upper3 = b3[estimator]
    yerr = [[mean1 - lower1, mean2 - lower2, mean3 - lower3], [upper1 - mean1, upper2 - mean2, upper3 - mean3]]
    
    ax.bar(
        xpos, [mean1, mean2, mean3], width / N, 
        yerr=yerr, capsize=3, label=estimator, 
        color=color_cycle[i], edgecolor='#ffffff', hatch=hatch_cycle[i]
    )
    ax.bar(xpos, [mean1, mean2, mean3], width / N, color='none', edgecolor='k', zorder=1)
    ax.set_ylabel('Average precision')
    ax.set_ylim(0, 0.45)
    ax.set_xticks(np.arange(len(groups)))
    ax.set_xticklabels(groups)
    ax.grid(True, axis='y')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1, 0, 0))
    
    if estimator in MLC:
        summary_perf.loc[('Strategy B', estimator)] = (
            print_ap(b1[estimator]),
            print_ap(b2[estimator]),
            print_ap(b3[estimator])
        )
    
plt.tight_layout()
axm.handlefig('2-performance-summary-stratb')

fig, ax = axm.setfig(1, 1, nseries=len(estimators), figsize=(7, 3))

for i, estimator in enumerate(estimators):
    offset = axm.get_barplot_offset(i, N)
    xpos = np.arange(len(groups)) + width * offset
    
    mean1, lower1, upper1 = c1[estimator]
    mean2, lower2, upper2 = c2[estimator]
    mean3, lower3, upper3 = c3[estimator]
    yerr = [[mean1 - lower1, mean2 - lower2, mean3 - lower3], [upper1 - mean1, upper2 - mean2, upper3 - mean3]]
    
    ax.bar(
        xpos, [mean1, mean2, mean3], width / N, 
        yerr=yerr, capsize=3, label=estimator, 
        color=color_cycle[i], edgecolor='#ffffff', hatch=hatch_cycle[i]
    )
    ax.bar(xpos, [mean1, mean2, mean3], width / N, color='none', edgecolor='k', zorder=1)
    ax.set_ylabel('Average precision')
    ax.set_ylim(0, 0.45)
    ax.set_xticks(np.arange(len(groups)))
    ax.set_xticklabels(groups)
    ax.grid(True, axis='y')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1, 0, 0))
    
    if estimator in MLC:
        summary_perf.loc[('Strategy C', estimator)] = (
            print_ap(c1[estimator]),
            print_ap(c2[estimator]),
            print_ap(c3[estimator])
        )
    
    else:
        summary_perf.loc[('CUS', estimator)] = (
            print_ap(c1[estimator]),
            print_ap(c2[estimator]),
            print_ap(c3[estimator])
        )

plt.tight_layout()
axm.handlefig('2-performance-summary-stratc')

In [ ]:
summary_perf.to_csv('data/summary-performances.csv')
summary_perf